# Data Extraction from Database Tables

In [2]:
import json
import pandas as pd
from sqlalchemy import create_engine, text

with open("../environment.json", "r") as f:
    env = json.load(f)

df = pd.DataFrame()

# データベースの接続情報
connection_config = {
    'user': env["DATABASE_USER"],
    'password': env["DATABASE_PASSWORD"],
    'host': env["DATABASE_HOST"],
    'database': 'songs'
}

engine = create_engine('postgresql://{user}:{password}@{host}/{database}'.format(**connection_config), echo=True)


## SQL Schema

```SQL
DROP TABLE IF EXISTS midi_features;
DROP TABLE IF EXISTS audio_features;
DROP TABLE IF EXISTS spotify_features;
DROP TABLE IF EXISTS song;

CREATE TABLE song (
    md5 TEXT NOT NULL UNIQUE,
    spotify_track_id TEXT NOT NULL UNIQUE,
    title TEXT,
    artist TEXT,
    publish_year smallint,
    PRIMARY KEY(md5, spotify_track_id)
);

CREATE TABLE midi_features (
    md5 TEXT REFERENCES song(md5),
    pitch_range INT,
    n_pitches_used INT,
    n_pitch_classes_used INT,
    polyphony FLOAT,
    polyphony_rate FLOAT,
    scale_consistency FLOAT,
    pitch_entropy FLOAT,
    pitch_class_entropy FLOAT,
    empty_beat_rate FLOAT,
    drum_in_duple_rate FLOAT,
    drum_in_triple_rate FLOAT,
    drum_pattern_consistency FLOAT
);

CREATE TABLE audio_features (
    spotify_track_id TEXT REFERENCES song(spotify_track_id),
    tempo FLOAT,
    zero_crossing_rate FLOAT[],
    harmonic_components FLOAT[],
    percussive_components FLOAT[],
    spectral_centroid FLOAT[],
    spectral_rolloff FLOAT[],
    chroma_frequencies FLOAT[]
);
```

In [6]:
q = text(
    f"SELECT song.title, song.artist, M.pitch_entropy, M.n_pitches_used, A.zero_crossing_rate FROM song INNER JOIN midi_features M on M.md5 = song.md5 INNER JOIN audio_features A on A.spotify_track_id = song.spotify_track_id;"
)
df = pd.read_sql_query(sql=q, con=engine)

df.head(20)

2022-07-12 14:40:41,724 INFO sqlalchemy.engine.Engine SELECT song.title, song.artist, M.pitch_entropy, M.n_pitches_used, A.zero_crossing_rate FROM song INNER JOIN midi_features M on M.md5 = song.md5 INNER JOIN audio_features A on A.spotify_track_id = song.spotify_track_id;
2022-07-12 14:40:41,726 INFO sqlalchemy.engine.Engine [cached since 18.25s ago] {}


,title,artist,pitch_entropy,n_pitches_used,zero_crossing_rate
0,Nun bitten wir den heiligen Geist a 5,Johann Walter,4.107287,21,"[0.0532170004826255, 0.0487998461631274, 0.049..."
1,La bella pargoletta,"Gabrieli, Andrea",4.073873,27,"[0.0888483349420849, 0.0896552244208494, 0.063..."
2,Vox dilecti mei,Giovanni Pierluigi da Palestrina,4.052571,25,"[0.0641495988175676, 0.0662686263272201, 0.070..."
3,Surge propera a 5,Giovanni Pierluigi da Palestrina,4.025558,26,"[0.0995603583494209, 0.0905130158059846, 0.096..."
4,Missa O magnum mysterium,Tomás Luis de Victoria,4.171363,27,"[0.0618986034025097, 0.0724843146718147, 0.068..."
5,Filiae Jerusalem,Andrea Gabrieli,4.147571,25,"[0.0652901785714286, 0.0518351079874517, 0.066..."
6,O magnum mysterium,Giovanni Gabrieli,4.371709,28,"[0.0886786619208494, 0.0834056014720077, 0.086..."
7,"Benedictus, Op. 34, No. 2",Edward Elgar,4.933580,64,"[0.0753687560328185, 0.0707913549710425, 0.067..."
8,"Mass No. 4 in C major, D452",Franz Schubert,4.876934,53,"[0.0928658150337838, 0.0832755188223938, 0.086..."
9,The Lost Chord,Arthur Sullivan,3.062451,16,"[0.221344111969112, 0.334023965371622, 0.23451..."
